In [1]:
!pip install -q langchain langchain-community langchain-google-genai google-generativeai chromadb pymupdf sentence-transformers tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.2 MB/s eta 0:00:00

In [2]:
from google.colab import userdata
from google.colab import files
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [3]:
# Securely get the API key from Colab secrets
try:
   GOOGLE_API_KEY = userdata.get('Divya_Key')
   print("Google API Key loaded successfully from Colab Secrets.")
except Exception as e:
   print(f"An error occurred while retrieving the API key: {e}")

Google API Key loaded successfully from Colab Secrets.


In [4]:
uploaded = files.upload()
documents = None
filename = next(iter(uploaded))
print(f'User uploaded file "{filename}"')


# Save the uploaded file to the Colab environment
with open(filename, 'wb') as f:
   f.write(uploaded[filename])

try:
   loader = PyMuPDFLoader(filename)
   documents = loader.load()
   print(f"Successfully loaded {len(documents)} pages from {filename}")
except Exception as e:
   print(f"An error occurred while loading the PDF: {e}")

Saving sr550_maintenance_manual.pdf to sr550_maintenance_manual.pdf
User uploaded file "sr550_maintenance_manual.pdf"
Successfully loaded 208 pages from sr550_maintenance_manual.pdf


In [5]:
# Step 2: Split the Document into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
splits = text_splitter.split_documents(documents)
print(f"Split document into {len(splits)} chunks.")

Split document into 298 chunks.


In [6]:
# Step 3: Create Embeddings via Sentence Transformers, Build VectorStore
try:
   # Initialize Sentence Transformer Embeddings model
   embeddings = SentenceTransformerEmbeddings(model_name="paraphrase-MiniLM-L3-v2")
   print("Sentence Transformer Embeddings model loaded.")


   # Create a vector store (Chroma) from the document chunks and embeddings
   print("\nCreating vector store from chunks...")
   vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
   print("Vector store created.")

except Exception as e:
   print(f"An error occurred during the RAG process: {e}")

/tmp/ipython-input-6-3107170395.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="paraphrase-MiniLM-L3-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence Transformer Embeddings model loaded.

Creating vector store from chunks...
Vector store created.


In [8]:
# Step 4: Set up the Retrieval Chain
llm = ChatGoogleGenerativeAI(
   model="gemini-2.0-flash-lite",
   google_api_key=GOOGLE_API_KEY
)


# Create a RetrievalQA chain and a prompt handler
qa_chain = RetrievalQA.from_chain_type(
   llm, retriever=vectorstore.as_retriever()
)


def qa_handler(query):
   response = qa_chain.invoke({"query": query})
   print(f"\nAsking question: {query}")
   print("\nGemini's Response:")
   print(response["result"])

In [9]:
# Step 5: Ask a question
qa_handler("What is this document about?")


Asking question: What is this document about?

Gemini's Response:
This document is a maintenance manual for the ThinkSystem SR550 server.
